**Chapter 8: Customizing LLMs and Their Output**

In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

from config import set_environment
set_environment()

## Fine-tuning

### Setup for fine-tuning

### Open-source models

### Commercial models

## Prompt Engineering

### Zero-shot prompting

In [2]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI()
prompt = PromptTemplate(
    input_variables=["text"],
    template="Classify the sentiment of this text: {text}"
)
chain = prompt | model
print(chain.invoke({"text": "I hated that movie, it was terrible!"}))

content='negative sentiment' additional_kwargs={} example=False


### Few-shot learning

In [3]:
examples = [{
"input": "I absolutely love the new update! Everything works seamlessly.",
"output": "Positive",
},{
"input": "It's okay, but I think it could use more features.",
"output": "Neutral",
}, {
"input": "I'm disappointed with the service, I expected much better performance.",
"output": "Negative"
}]

In [4]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

example_prompt = PromptTemplate(
    template="{input} -> {output}",
    input_variables=["input", "output"],
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)
model = ChatOpenAI()
chain = prompt | model
print(chain.invoke({"input": " This is an excellent book with high quality explanations."}))

content='Positive' additional_kwargs={} example=False


### Chain-of-Thought prompting

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

reasoning_prompt = "{question}\nLet's think step by step!"
prompt = PromptTemplate(
    template=reasoning_prompt,
    input_variables=["question"]
)
model = ChatOpenAI()
chain = prompt | model
print(chain.invoke({
    "question": "There were 5 apples originally. I ate 2 apples. My friend gave me 3 apples. How many apples do I have now?",
}))

content='1. You start with 5 apples.\n2. You eat 2 apples, so 5 - 2 = 3 apples.\n3. Your friend gives you 3 apples, so 3 + 3 = 6 apples.\n\nSo, you now have 6 apples.' additional_kwargs={} example=False


In [7]:
examples = [{
    "input": "I absolutely love the new update! Everything works seamlessly.",
    "output": "Love and absolute works seamlessly are examples of positive sentiment. Therefore, the sentiment is positive",
    },{
    "input": "It's okay, but I think it could use more features.",
    "output": "It's okay is not an endorsement. The customer further thinks it should be extended. Therefore, the sentiment is neutral",
    }, {
    "input": "I'm disappointed with the service, I expected much better performance.",
    "output": "The customer is disappointed and expected more. This is negative"
}]

In [8]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

example_prompt = PromptTemplate(
    template="{input} -> {output}",
    input_variables=["input", "output"],
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)
model = ChatOpenAI()
chain = prompt | model
print(chain.invoke({"input": " This is an excellent book with high quality explanations."}))

content='Answer: This is a positive sentiment.' additional_kwargs={} example=False


### Self-consistency

In [11]:
from langchain import PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI

solutions_template = """
Generate {num_solutions} distinct answers to this question: {question}

Solutions:
"""
solutions_prompt = PromptTemplate(
    template=solutions_template,
    input_variables=["question", "num_solutions"]
)
solutions_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=solutions_prompt,
    output_key="solutions"
)

In [12]:
consistency_template = """
For each answer in {solutions}, count the number of times it occurs. Finally, choose the answer that occurs most.

Most frequent soution:
"""
consistency_prompt = PromptTemplate(
    template=consistency_template,
    input_variables=["solutions"]
)
consistency_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=consistency_prompt,
    output_key="best_solution"
)

In [13]:
from langchain.chains import SequentialChain
answer_chain = SequentialChain(
    chains=[solutions_chain, consistency_chain],
    input_variables=["question", "num_solutions"],
    output_variables=["best_solution"]
)

In [14]:
print(answer_chain.run(
    question="Which year was the Declaration of Independence of the United States signed?",
    num_solutions="5"
))

1776 occurs 5 times, so the most frequent solution is 1776.


### Tree-of-thought

In [15]:
solutions_template = """
Generate {num_solutions} distinct solutions for {problem}. Consider factors like {factors}.

Solutions:
"""
solutions_prompt = PromptTemplate(
    template=solutions_template,
    input_variables=["problem", "factors", "num_solutions"]
)

In [16]:
evaluation_template = """
Evaluate each solution in {solutions} by analyzing pros, cons,
feasibility, and probability of success.

Evaluations:
"""
evaluation_prompt = PromptTemplate(
    template=evaluation_template,
    input_variables=["solutions"]
)

In [17]:
reasoning_template = """
For the most promising solutions in {evaluations}, explain scenarios,
implementation strategies, partnerships needed, and handling potential
obstacles.

Enhanced Reasoning:
"""
reasoning_prompt = PromptTemplate(
    template=reasoning_template,
    input_variables=["evaluations"]
)

In [18]:
ranking_template = """
Based on the evaluations and reasoning, rank the solutions in {enhanced_reasoning} from most to least promising.

Ranked Solutions:
"""
ranking_prompt = PromptTemplate(
    template=ranking_template,
    input_variables=["enhanced_reasoning"]
)

In [19]:
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI

solutions_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=solutions_prompt,
    output_key="solutions"
)
evalutation_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=evaluation_prompt,
    output_key="evaluations"
)
reasoning_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=reasoning_prompt,
    output_key="enhanced_reasoning"
)
ranking_chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=ranking_prompt,
    output_key="ranked_solutions"
)

In [20]:
from langchain.chains import SequentialChain
tot_chain = SequentialChain(
    chains=[solutions_chain, evalutation_chain, reasoning_chain, ranking_chain],
    input_variables=["problem", "factors", "num_solutions"],
    output_variables=["ranked_solutions"]
)


In [21]:
print(tot_chain.run(
    problem="Prompt engineering",
    factors="Requirements for high task performance, low token use, and few calls to the LLM",
    num_solutions=3
))

1. Implementing a comprehensive training program for engineers
2. Developing an AI-powered tool to automate routine tasks
3. Introducing a performance-based incentive program
